In [1]:
import pandas
import re

data = pandas.read_excel("AMAM eMuseum data export 9.26.24.xlsx")
data["Dated"]

0             19th century
1                     1856
2                  1855–56
3                 ca. 1853
4       early 20th century
               ...        
2144                  2022
2145              ca. 1883
2146                  1983
2147                  2024
2148                  1991
Name: Dated, Length: 2149, dtype: object

In [110]:
def hyphenate(entry):
    entry = entry.replace("—", "-")
    entry = entry.replace("–", "-")
    return entry


def parse(items):
    dates = pandas.DataFrame(columns=["ObjectID", "Start Year", "End Year", "Approximate"])
    for row in items.itertuples():
        id = row.ObjectID
        entry = row.Dated
        m = re.fullmatch(r"(\d+)(?:st|nd|rd|th) century(?: CE)?", entry)
        if m:
            century = int(m.group(1)) * 100
            dates.loc[len(dates)] = [id, century-99, century, False]
            continue
        m = re.fullmatch(r"(?:\w+\s+)?(\d+)", entry)
        if m:
            year = int(m.group(1))
            dates.loc[len(dates)] = [id, year, year, False]
            continue
        m = re.fullmatch(r"(?:printed )?(\d{2,})-(\d{2})(?: \(restrike\))?", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 100) + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa |ca\.|after )(\d+)", entry)
        if m:
            year = int(m.group(1))
            dates.loc[len(dates)] = [id, year, year, True]
            continue
        m = re.fullmatch(r"(?:Probably |)early (\d{1,2})(?:st|nd|rd|th) century(?: CE)?", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) (?:C|c)entur(?:y|ies)", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 99
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th) century(?: CE|)", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 33
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 9
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"second half (\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 50
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th)\s*(?:-|to )early (\d{1,2})(?:st|nd|rd|th)\s*century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 33
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{2,})-(\d{2})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - start_year % 100 + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th)(?:-| to )early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 66
            end_year = (int(m.group(2)) * 100) - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{3,})-(\d{3,})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(\d+) cast of (?:ca.|circa) (\d+) original", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, True]
            continue
        m = re.fullmatch(r"early (\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = int(m.group(2)) * 100 - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) +century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 33
            end_year = int(m.group(2)) * 100
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 66
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|repainted in|printed(?: in)?) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, False]
            continue
        m = re.fullmatch(r"mid-(\d{1,2})(?:st|nd|rd|th) century CE \(after a (\d{1,2})(?:st|nd|rd|th) century BCE Greek original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 66
            start_year2 = start_year1 + 33
            end_year1 = -(int(m.group(2)) * 100)
            end_year2 = end_year1 + 99
            dates.loc[len(dates)] = [id, start_year1, start_year2, True]
            dates.loc[len(dates)] = [id, end_year1, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-early (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = int(m.group(2)) * 100 - 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) century CE \(after a (\d{1,2})(?:st|nd|rd|th) century BCE Greek original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 99
            start_year2 = int(m.group(2)) * 100
            end_year1 = -(int(m.group(3)) * 100)
            end_year2 = end_year1 + 99
            dates.loc[len(dates)] = [id, start_year1, start_year2, False]
            dates.loc[len(dates)] = [id, end_year1, end_year2, False]
            continue
        m = re.fullmatch(r"ca\. (\d+)-(\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            end_year = -int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{2,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 6
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d+)-(\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            end_year = -int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"ca\. (\d+) BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1))
            dates.loc[len(dates)] = [id, start_year, start_year, True]
            continue
        m = re.fullmatch(r"(\d{1,})-(\d{1})", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 10) + int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) century \(imitation of (\d+) original\)", entry)
        if m:
            start_year1 = int(m.group(1)) * 100 - 99
            end_year1 = int(m.group(2)) * 100
            start_year2 = int(m.group(3))
            dates.loc[len(dates)] = [id, start_year1, end_year1, False]
            dates.loc[len(dates)] = [id, start_year2, start_year2, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{1,2})(?:st|nd|rd|th) century BCE", entry)
        if m:
            start_year = -int(m.group(1)) * 100
            end_year = start_year + 99
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)-(\d{1,2})(?:st|nd|rd|th) (?:C|c)entury BC(?:E|)", entry)
        if m:
            start_year = -int(m.group(1)) * 100
            end_year = -int(m.group(2)) * 100 - 99
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"second half (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 50
            end_year = start_year + 50
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"first half (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = start_year + 49
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early to mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = start_year + 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"late (\d{1,3}0)s-early (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 6
            end_year = int(m.group(2)) + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid- to late (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 3
            end_year = start_year + 6
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early (\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"mid-(\d{1,3}0)s", entry)
        if m:
            start_year = int(m.group(1)) + 3
            end_year = start_year + 3
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"early (\d{1,2})(?:st|nd|rd|th) century, original (\d{2,})-(\d{2})", entry)
        if m:
            end_year1 = int(m.group(1)) * 100
            start_year1 = end_year1 - 99
            start_year2 = int(m.group(2))
            end_year2 = start_year2 - (start_year2 % 100) + int(m.group(3))
            dates.loc[len(dates)] = [id, start_year1, end_year1, True]
            dates.loc[len(dates)] = [id, start_year2, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,3}0)s-(\d0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year - (start_year % 100) + int(m.group(2)) + 9
            dates.loc[len(dates)] = [id, start_year, end_year, False]
            continue
        m = re.fullmatch(r"ca\. (\d{2,})-(\d{2}).+,.+(\d{2,})-(\d{2})", entry)
        if m:
            start_year1 = int(m.group(1))
            end_year1 = start_year1 - (start_year1 % 100) + int(m.group(2))
            start_year2 = int(m.group(3))
            end_year2 = start_year2 - (start_year2 % 100) + int(m.group(4))
            dates.loc[len(dates)] = [id, start_year1, end_year1, True]
            dates.loc[len(dates)] = [id, start_year2, end_year2, False]
            continue
        m = re.fullmatch(r"(\d{1,2})(?:st|nd|rd|th)(?:-| to )mid-(\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 99
            end_year = (int(m.group(2)) * 100) - 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa |ca\.)(\d+)-(\d+)", entry)
        if m:
            start_year = int(m.group(1))
            end_year = int(m.group(2))
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "20th century pastiche using components from second quarter 16th century":
            dates.loc[len(dates)] = [id, 1901, 2000, False]
            dates.loc[len(dates)] = [id, 1525, 1550, False]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|printed) (?:ca\. |circa )(\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year2, True]
            continue
        m = re.fullmatch(r"\w* ?\w* \d{1,2}, (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            dates.loc[len(dates)] = [id, year1, year1, False]
            continue
        m = re.fullmatch(r"(\d{2,})-(\d{2}), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = year1 - (year1 % 100) + int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        if entry == "1970 (enlarged refabrication of 1964 project)":
            dates.loc[len(dates)] = [id, 1964, 1964, False]
            dates.loc[len(dates)] = [id, 1970, 1970, False]
            continue
        m = re.fullmatch(r"(\d+)-(\d+), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        if entry == "ca. 1970 (modern print from 1896 negative)":
            dates.loc[len(dates)] = [id, 1970, 1970, True]
            dates.loc[len(dates)] = [id, 1896, 1896, False]
            continue
        m = re.fullmatch(r"(\d+), (?:published|reworked in|printed) (\d{2,})-(\d{2})", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = year2 - (year2 % 100) + int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year1, False]
            dates.loc[len(dates)] = [id, year2, year3, False]
            continue
        if entry == "negative ca. 1865, printed early 1870s":
            dates.loc[len(dates)] = [id, 1865, 1865, True]
            dates.loc[len(dates)] = [id, 1870, 1873, True]
            continue
        m = re.fullmatch(r"[Ee]arly (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = int(m.group(1)) * 100 - 99
            end_year = start_year + 33
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{3}0)s", entry)
        if m:
            start_year = int(m.group(1))
            end_year = start_year + 9
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "n.d.":
            dates.loc[len(dates)] = [id, 0, 0, True]
            continue
        if entry == "1931, printed later":
            dates.loc[len(dates)] = [id, 1931, 1931, False]
            continue
        m = re.fullmatch(r"(?:ca\. |circa )(\d{2,})-(\d{2}), (?:published|reworked in|printed) (\d+)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = year1 - (year1 % 100) + int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, True]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue
        m = re.fullmatch(r"mid-? to late (\d{1,2})(?:st|nd|rd|th) century", entry)
        if m:
            start_year = (int(m.group(1)) * 100) - 66
            end_year = start_year + 66
            dates.loc[len(dates)] = [id, start_year, end_year, True]
            continue
        if entry == "late 19th-early 20th century (heCE replaced 1930-59)":
            dates.loc[len(dates)] = [id, 1867, 1933, True]
            dates.loc[len(dates)] = [id, 1930, 1959, False]
            continue
        if entry == "2009- (letter transcribed in 2018)":
            dates.loc[len(dates)] = [id, 2009, 2009, False]
            dates.loc[len(dates)] = [id, 2018, 2018, False]
            continue
        m = re.fullmatch(r"(\d+)(?:st|nd|rd|th) century BCE", entry)
        if m:
            century = -(int(m.group(1)) * 100)
            dates.loc[len(dates)] = [id, century, century + 99, False]
            continue
        m = re.fullmatch(r"(\d+)-(\d+),? \((?:published|reworked in|printed) (\d+)\)", entry)
        if m:
            year1 = int(m.group(1))
            year2 = int(m.group(2))
            year3 = int(m.group(3))
            dates.loc[len(dates)] = [id, year1, year2, False]
            dates.loc[len(dates)] = [id, year3, year3, False]
            continue

        print(dates)
        raise ValueError(f"Failed to parse {entry}")
    return dates

data["Dated"] = data["Dated"].map(hyphenate)
dates_pre = data[["ObjectID", "Dated"]]
dates = parse(dates_pre)
dates

,ObjectID,Start Year,End Year,Approximate
0,10739,1801,1900,False
1,4249,1856,1856,False
2,2727,1855,1856,False
3,12652,1853,1853,True
4,2548,1901,1934,True
...,...,...,...,...
2226,40978,2022,2022,False
2227,40987,1883,1883,True
2228,41009,1983,1983,False
2229,41049,2024,2024,False


In [112]:
def dimensions(items):
    dims = pandas.DataFrame(columns=["ObjectID", "Type", "Length", "Width", "Height", "Duration"])
    for row in items.itertuples():
        id = row.ObjectID
        entry = row.Dimension
        listings = entry.splitlines()
        for listing in listings:
            m = re.match(r"(\w+):", listing)
            if not m:
                raise ValueError(f"Failed to parse {listing}")
            type = m.group(1)
            if type == "Duration":
                n = re.match(r"(\d+\.?\d*)", listing)
                if not n:
                    raise ValueError(f"Failed to parse {listing}")
                duration = float(n.group(1))
                dims.loc[len(dims)] = [id, type, None, None, None, duration]
                continue
            cm_section = listing.split("(")[1]
            n = re.findall(r"(\d+\.?\d*)", cm_section)
            # TODO: finish getting dimensions extracted
            # Remember that one object that has 4 spatial dimensions? Gotta deal with that.


dims = data[["ObjectID", "Dimension"]]
dim_table = dimensions(dims)

['Overall: 8 1/2 × 5 in. (21.6 × 12.7 cm)']
['Image/Sheet: 2 × 13 3/16 in. (5.1 × 33.5 cm)']
['Image: 6 1/4 × 5 in. (15.9 × 12.7 cm)', 'Plate: 6 1/2 × 5 1/4 in. (16.5 × 13.3 cm)', 'Sheet: 8 × 6 1/2 in. (20.3 × 16.5 cm)']
['Image: 15 × 22 1/4 in. (38.1 × 56.5 cm)', 'Sheet: 15 1/2 × 22 1/4 in. (39.4 × 56.5 cm)']
['Overall: 5 13/16 × 2 1/4 × 3 3/4 in. (14.8 × 5.7 × 9.5 cm)']
['Overall: 12 × 7 5/8 × 9 1/2 in. (30.5 × 19.4 × 24.1 cm)']
['Overall: 32 in. (81.3 cm)']
['Overall: 7 × 3 in. (17.8 × 7.6 cm)']
['Overall: 3 × 5 in. (7.6 × 12.7 cm)']
['Overall: 4 3/4 × 4 in. (12.1 × 10.2 cm)', 'Frame: 12 1/4 × 11 3/8 × 2 1/4 in. (31.1 × 28.9 × 5.7 cm)']
['Overall: 1 1/2 × 2 × 2 3/8 in. (3.8 × 5.1 × 6 cm)']
['Overall: 11 × 10 in. (27.9 × 25.4 cm)']
['Overall: 16 5/8 × 8 7/16 in. (42.2 × 21.4 cm)']
['Overall: 18 × 9 in. (45.7 × 22.9 cm)']
['Overall: 11 × 8 in. (27.9 × 20.3 cm)']
['Overall: 11 5/8 × 9 5/8 × 5 1/2 in. (29.5 × 24.4 × 14 cm)']
['Overall: 15 1/2 × 24 9/16 in. (39.4 × 62.3 cm)', 'Frame: 26 

ValueError: Columns must be same length as key